PART 02 파이썬 분석

> CHAPTER 03 제2유형: 데이터 분석
> >SECTION 02 지도학습 - 분류
> > >**01 의사결정나무를 이용한 분류 문제 해결**

In [1]:
import numpy as np      # 넘파이 패키지 임포트
import pandas as pd     # 판다스 패키지 임포트
import sklearn          # 싸이킷런 패키지 임포트

# 의사결정나무 분류모델을 위한 패키지 임포트
from sklearn.tree import DecisionTreeClassifier
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [2]:
# 깃허브에 공개되어 있는 csv 파일을 읽어와서 데이터프레임 df로 넣는다.
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [3]:
df

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  27.0      0   
887                       Graham, Miss. Margaret Edith  female  19.0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   NaN      1   
889                              Behr, Mr. Karl Howell    male  26.0      0   
890                                Dooley, Mr. Patrick    male  32.0      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [4]:
df.info()         # 데이터프레임 결측값 유무 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.describe()       # 데이터프레임의 기술통계 보여주기

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [6]:
# Age 컬럼의 결측값을 평균으로 대치한다.
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

# Embarked 컬럼의 결측값을 최빈값으로 대치한다.
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

In [7]:
# Sex 컬럼의 값을 1과 0으로 레이블인코딩 한다.
from sklearn.preprocessing import LabelEncoder
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

# Embarked 컬럼의 값을 레이블인코딩 한다.
from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

In [8]:
# SibSp, Parch의 값을 더해서 FamilySize 컬럼(파생변수)을 생성한다.
df["FamilySize"] = df["SibSp"]+df["Parch"]

In [9]:
df

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    1  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.000000      1   
2                               Heikkinen, Miss. Laina    0  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.000000      1   
4                             Allen, Mr. William Henry    1  35.000000      0   
..                                                 ...  ...        ...    ...   
886                              Montvila, Rev. Juozas    1  27.000000      0   
887                       Graham, Miss. Margaret Edith    0  19.000000      0   
888           Johnston, Miss. Catherine Helen "Carrie"    0  29.699118      1   
889                              Behr, Mr. Karl Howell    1  26.000000      0   
890                                Dooley, Mr. Patrick    1  32.000000      0   

     Parch            Ticket     Fare Cabin  Embarked  FamilySize  
0        0         A/5 21171   7.2500   NaN         2           1  
1        0          PC 17599  71.2833   C85         0           1  
2        0  STON/O2. 3101282   7.9250   NaN         2           0  
3        0            113803  53.1000  C123         2           1  
4        0            373450   8.0500   NaN         2           0  
..     ...               ...      ...   ...       ...         ...  
886      0            211536  13.0000   NaN         2           0  
887      0            112053  30.0000   B42         2           0  
888      2        W./C. 6607  23.4500   NaN         2           3  
889      0            111369  30.0000  C148         0           0  
890      0            370376   7.7500   NaN         1           0  

[891 rows x 13 columns]

In [10]:
# 분석 데이터셋 준비
# X는 독립변수(설명변수), y는 종속변수(목표변수)
X = df[["Pclass","Sex","Age","Fare","Embarked","FamilySize"]]
y = df["Survived"]

In [11]:
X

Pclass  Sex        Age     Fare  Embarked  FamilySize
0         3    1  22.000000   7.2500         2           1
1         1    0  38.000000  71.2833         0           1
2         3    0  26.000000   7.9250         2           0
3         1    0  35.000000  53.1000         2           1
4         3    1  35.000000   8.0500         2           0
..      ...  ...        ...      ...       ...         ...
886       2    1  27.000000  13.0000         2           0
887       1    0  19.000000  30.0000         2           0
888       3    0  29.699118  23.4500         2           3
889       1    1  26.000000  30.0000         0           0
890       3    1  32.000000   7.7500         1           0

[891 rows x 6 columns]

In [12]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [13]:
# 분석 데이터셋 분할(8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [14]:
print(X_train.shape)    # 학습 데이터(독립변수)
print(X_test.shape)     # 테스트 데이터(독립변수)
print(y_train.shape)    # 학습 데이터(종속변수)
print(y_test.shape)     # 테스트 데이터(종속변수)

(712, 6)
(179, 6)
(712,)
(179,)


In [15]:
# DecisionTreeClassifier 객체 생성
dt = DecisionTreeClassifier(random_state=11)
dt.fit(X_train, y_train)      # 학습 수행

DecisionTreeClassifier(random_state=11)

In [16]:
# 학습이 완료된 dt객체에서 테스트 데이터셋으로 예측 수행
pred = dt.predict(X_test)

In [17]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.7877094972067039


>>>**02 KNN을 이용한 분류 문제 해결**

In [36]:
import numpy as np      # 넘파이 패키지 임포트
import pandas as pd     # 판다스 패키지 임포트
import sklearn          # 싸이킷런 패키지 임포트

# KNN 분류모델을 위한 패키지 임포트
from sklearn.neighbors import KNeighborsClassifier
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [37]:
# 깃허브에 있는 csv 파일을 읽어와서 데이터프레임 df로 넣는다.
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

In [20]:
df

sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]

In [21]:
df.info()         # 데이터프레임 결측값 유무 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [22]:
df.describe()       # 데이터프레임의 기술통계 보여주기

sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000

In [38]:
# 각 독립변수별 Min-Max 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])

In [39]:
# 분석 데이터셋 준비
# X는 독립변수(설명변수), y는 종속변수(목표변수)
X = df[["sepal_length","sepal_width","petal_length","petal_width"]]
y = df["species"]

# 분석 데이터셋 분할(8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [25]:
print(X_train.shape)     # 학습 데이터(독립변수)
print(X_test.shape)      # 테스트 데이터(독립변수)
print(y_train.shape)     # 학습 데이터(종속변수)
print(y_test.shape)      # 테스트 데이터(종속변수)

(120, 4)
(30, 4)
(120,)
(30,)


In [26]:
# KNeighborsClassifier 객체 생성
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)      # 학습 수행

KNeighborsClassifier(n_neighbors=3)

In [27]:
# 학습이 완료된 dt객체에서 테스트 데이터셋으로 예측 수행
pred = knn.predict(X_test)

In [28]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.9333333333333333


In [29]:
# 4) 추가 작업 해보기

In [40]:
# KNeighborsClassifier 객체 생성
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train) # 학습 수행

# 학습이 완료된 dt 객체에서 테스트 데이터셋으로 예측 수행
pred = knn.predict(X_test)

In [41]:
# 모델 성능 평가 - Confusion Matrix 계산
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

[[ 9  0  0]
 [ 0 10  0]
 [ 0  2  9]]


In [42]:
# 모델 성능 평가 - 평가지표 계산
from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       0.83      1.00      0.91        10
   virginica       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30



>>>**03 SVM을 이용한 분류 문제 해결**

In [72]:
import numpy as np      # 넘파이 패키지 임포트
import pandas as pd     # 판다스 패키지 임포트
import sklearn          # 싸이킷런 패키지 임포트

# 서포트벡터머신 분류모델을 위한 패키지 임포트
from sklearn import svm
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [73]:
# 깃허브에 공개되어 있는 csv 파일을 읽어와서 데이터프레임 df로 넣는다.
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [74]:
df

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  27.0      0   
887                       Graham, Miss. Margaret Edith  female  19.0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   NaN      1   
889                              Behr, Mr. Karl Howell    male  26.0      0   
890                                Dooley, Mr. Patrick    male  32.0      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [75]:
# Age 컬럼의 결측값을 평균으로 대치한다.
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

# Embarked 컬럼의 결측값을 최빈값으로 대치한다.
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

# SibSp, Parch의 값을 더해서 FamilySize 컬럼(파생변수)을 생성한다.
df["FamilySize"] = df["SibSp"]+df["Parch"]

In [76]:
# Sex 컬럼의 값을 1과 0으로 원-핫 인코딩 한다.
onehot_sex = pd.get_dummies(df["Sex"])
df = pd.concat([df, onehot_sex], axis=1)

# Embarked 컬럼의 값을 원-핫 인코딩 한다.
onehot_embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, onehot_embarked], axis=1)

In [77]:
df

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex        Age  \
0                              Braund, Mr. Owen Harris    male  22.000000   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.000000   
2                               Heikkinen, Miss. Laina  female  26.000000   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.000000   
4                             Allen, Mr. William Henry    male  35.000000   
..                                                 ...     ...        ...   
886                              Montvila, Rev. Juozas    male  27.000000   
887                       Graham, Miss. Margaret Edith  female  19.000000   
888           Johnston, Miss. Catherine Helen "Carrie"  female  29.699118   
889                              Behr, Mr. Karl Howell    male  26.000000   
890                                Dooley, Mr. Patrick    male  32.000000   

     SibSp  Parch            Ticket     Fare Cabin Embarked  FamilySize  \
0        1      0         A/5 21171   7.2500   NaN        S           1   
1        1      0          PC 17599  71.2833   C85        C           1   
2        0      0  STON/O2. 3101282   7.9250   NaN        S           0   
3        1      0            113803  53.1000  C123        S           1   
4        0      0            373450   8.0500   NaN        S           0   
..     ...    ...               ...      ...   ...      ...         ...   
886      0      0            211536  13.0000   NaN        S           0   
887      0      0            112053  30.0000   B42        S           0   
888      1      2        W./C. 6607  23.4500   NaN        S           3   
889      0      0            111369  30.0000  C148        C           0   
890      0      0            370376   7.7500   NaN        Q           0   

     female  male  C  Q  S  
0         0     1  0  0  1  
1         1     0  1  0  0  
2         1     0  0  0  1  
3         1     0  0  0  1  
4         0     1  0  0  1  
..      ...   ... .. .. ..  
886       0     1  0  0  1  
887       1     0  0  0  1  
888       1     0  0  0  1  
889       0     1  1  0  0  
890       0     1  0  1  0  

[891 rows x 18 columns]

In [78]:
# 분석 데이터셋 준비
# X는 독립변수(설명변수), y는 종속변수(목표변수)
X = df[["Pclass","Age","Fare","FamilySize","female","male","C","Q","S"]]
y = df["Survived"]

In [79]:
# 분석 데이터셋 분할(7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [80]:
print(X_train.shape)     # 학습 데이터(독립변수)
print(X_test.shape)      # 테스트 데이터(독립변수)
print(y_train.shape)     # 학습 데이터(종속변수)
print(y_test.shape)      # 테스트 데이터(종속변수)

(623, 9)
(268, 9)
(623,)
(268,)


In [81]:
# SVM 객체 생성 - 커널 rbf 적용
sv = svm.SVC(kernel='rbf')
sv.fit(X_train, y_train)      # 학습 수행

SVC()

In [82]:
# 학습이 완료된 dt 객체에서 테스트 데이터셋으로 예측 수행
pred = sv.predict(X_test)

In [83]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.7238805970149254


In [84]:
# 모델 성능 평가 - Confusion Matrix 계산
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

[[167   7]
 [ 67  27]]


In [85]:
# 모델 성능 평가 - 평가지표 계산
from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

              precision    recall  f1-score   support

           0       0.71      0.96      0.82       174
           1       0.79      0.29      0.42        94

    accuracy                           0.72       268
   macro avg       0.75      0.62      0.62       268
weighted avg       0.74      0.72      0.68       268



>>>**04 로지스틱 회귀를 이용한 분류 문제 해결**

In [8]:
import numpy as np      # 넘파이 패키지 임포트
import pandas as pd     # 판다스 패키지 임포트
import sklearn          # 싸이킷런 패키지 임포트

# 로지스틱 회귀 분류모델을 위한 패키지 임포트
from sklearn.linear_model import LogisticRegression
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [9]:
# 깃허브에 있는 csv 파일을 읽어와서 데이터프레임 df로 넣는다.
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

In [10]:
df

sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [12]:
df.describe()

sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000

In [14]:
# 각 독립변수별 Min-Max 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])

In [15]:
# 분석 데이터셋 준비
# X는 독립변수(설명변수), y는 종속변수(목표변수)
X = df[["sepal_length","sepal_width","petal_length","petal_width"]]
y = df["species"]

# 분석 데이터셋 분할(8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [16]:
print(X_train.shape)  # 학습 데이터(독립변수)
print(X_test.shape)   # 테스트 데이터(독립변수)
print(y_train.shape)  # 학습 데이터(종속변수)
print(y_test.shape)   # 테스트 데이터(종속변수)

(120, 4)
(30, 4)
(120,)
(30,)


In [17]:
# LogisticRegression 객체 생성
lr = LogisticRegression()
lr.fit(X_train, y_train)      # 학습 수행

LogisticRegression()

In [18]:
# 학습이 완료된 dt객체에서 테스트 데이터셋으로 예측 수행
pred = lr.predict(X_test)

In [19]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.8333333333333334


>>>**05 랜덤 포레스트를 이용한 분류 문제 해결**

In [20]:
import numpy as np    # 넘파이 패키지 임포트
import pandas as pd   # 판다스 패키지 임포트
import sklearn        # 싸이킷런 패키지 임포트

# 랜덤 포레스트 분류모델을 위한 패키지 임포트
from sklearn.ensemble import RandomForestClassifier
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [21]:
# 깃허브에 공개되어 있는 csv 파일을 읽어와서 데이터프레임 df로 넣는다.
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [22]:
df

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  27.0      0   
887                       Graham, Miss. Margaret Edith  female  19.0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   NaN      1   
889                              Behr, Mr. Karl Howell    male  26.0      0   
890                                Dooley, Mr. Patrick    male  32.0      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [23]:
df.info()         # 데이터프레임 결측값 유무 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
df.describe()       # 데이터프레임의 기술통계 보여주기

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [25]:
# Age 컬럼의 결측값을 평균으로 대치한다.
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

# Embarked 컬럼의 결측값을 최빈값으로 대치한다.
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

In [26]:
# Sex 컬럼의 값을 1과 0으로 레이블인코딩 한다.
from sklearn.preprocessing import LabelEncoder
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

# Embarked 컬럼의 값을 레이블인코딩 한다.
from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

In [27]:
# SibSp, Parch의 값을 더해서 FamilySize 컬럼(파생변수)를 생성한다.
df["FamilySize"] = df["SibSp"]+df["Parch"]

In [28]:
# 분석 데이터셋 준비
# X는 독립변수(설명변수), y는 종속변수(목표변수)
X = df[["Pclass","Sex","Age","Fare","Embarked","FamilySize"]]
y = df["Survived"]

In [29]:
X

Pclass  Sex        Age     Fare  Embarked  FamilySize
0         3    1  22.000000   7.2500         2           1
1         1    0  38.000000  71.2833         0           1
2         3    0  26.000000   7.9250         2           0
3         1    0  35.000000  53.1000         2           1
4         3    1  35.000000   8.0500         2           0
..      ...  ...        ...      ...       ...         ...
886       2    1  27.000000  13.0000         2           0
887       1    0  19.000000  30.0000         2           0
888       3    0  29.699118  23.4500         2           3
889       1    1  26.000000  30.0000         0           0
890       3    1  32.000000   7.7500         1           0

[891 rows x 6 columns]

In [30]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [31]:
# 분석 데이터셋 분할(8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [32]:
print(X_train.shape)    # 학습 데이터(독립변수)
print(X_test.shape)     # 테스트 데이터(독립변수)
print(y_train.shape)    # 학습 데이터(종속변수)
print(y_test.shape)     # 테스트 데이터(종속변수)

(712, 6)
(179, 6)
(712,)
(179,)


In [33]:
# RandomForestClassifier 객체 생성
rf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=20)
rf.fit(X_train, y_train)      # 학습 수행

RandomForestClassifier(max_depth=3, n_estimators=50, random_state=20)

In [34]:
# 학습이 완료된 rf객체에서 테스트 데이터셋으로 예측 수행
pred = rf.predict(X_test)

In [35]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.8603351955307262
